In [52]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
import urllib3
urllib3.disable_warnings()

In [53]:
path = r"https://www.nbp.pl/home.aspx?f=/dzienne/stopy_archiwum.htm"
page = requests.get(path, verify=False)
soup = BeautifulSoup(page.content, 'html.parser')

In [61]:
def download_data(soup):
    html_data = soup.find_all('tr', {'valign': 'middle'})

    rows = []
    for tr in html_data:
        td = tr.find_all('td')
        row = [tr.text.replace(" ", "").replace(",", ".") for tr in td]

        # wyrzucenie pustych wierszy i wierszy z samym rokiem
        try:
            int(row[0])
        except (IndexError, TypeError, ValueError):
            if row:
                rows.append(row)

    nbp_df = pd.DataFrame(rows, 
                          columns=['obowiązuje_od', 'stopa_referencyjna', 'stopa_lombartowa', 'stopa_depozytowa', 
                                   'stopa_redyskontowa_weksli', 'stopa_dyskontowa_weksli'])
    nbp_df.replace("*", None, inplace=True)
    
    # stringi na liczby i datę
    nbp_df.iloc[:,1:] = nbp_df.iloc[:,1:].astype(float)
    nbp_df['obowiązuje_od'] = pd.to_datetime(nbp_df['obowiązuje_od'])
    nbp_df['obowiązuje_od'] = nbp_df['obowiązuje_od'].dt.date
    
    return nbp_df

In [62]:
path = r"https://www.nbp.pl/home.aspx?f=/dzienne/stopy_archiwum.htm"
page = requests.get(path, verify=False)
soup = BeautifulSoup(page.content, 'html.parser')

nbp_df = download_data(soup)

In [63]:
nbp_df.to_excel('stopy_nbp.xlsx', index=False)

In [60]:
nbp_df['obowiązuje_od'].dt.date

0     1998-02-26
1     1998-04-23
2     1998-05-21
3     1998-07-17
4     1998-09-10
         ...    
81    2022-03-09
82    2022-04-07
83    2022-05-06
84    2022-06-09
85    2022-07-08
Name: obowiązuje_od, Length: 86, dtype: object